In [1]:
import sys
sys.path.append('./Ray_finrl/findrl_ray/finenv')
from finenv.env_stocktrading import StockTradingEnv
from finenv.preprocessors import FeatureEngineer, data_split
import pandas as pd
import pickle 
# load the DataFrame from a pickle file
#df = pd.read_pickle('dataset/processed.pkl')

trade = pd.read_csv('dataset/trade_data.csv')
trade = trade.set_index(trade.columns[0])
trade.index.names = ['']

TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-01-01'
TRADE_START_DATE = '2021-01-02'
TRADE_END_DATE = '2023-03-24'

In [2]:
INDICATORS = ['macd','boll_ub','boll_lb','rsi_30','cci_30','dx_30','close_30_sma','close_60_sma']

In [3]:
import psutil
import ray
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total
from ray.tune.registry import register_env
from gymnasium.wrappers import EnvCompatibility

In [4]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
,,,,,,,,,,,,,,,,
0,2019-05-13,AAL,33.160000,33.200001,31.980000,31.806324,7040900.0,0.0,0.068303,34.569498,32.334917,45.215431,-214.170585,27.605727,33.410860,32.926946
0,2019-05-13,AAPL,46.927502,47.369999,45.712502,45.109749,229722400.0,0.0,0.197157,51.926065,46.557557,45.444855,-217.008364,36.029204,48.727347,46.152760
0,2019-05-13,ADBE,270.570007,274.190002,267.220001,267.700012,2698300.0,0.0,1.817067,289.657626,265.629380,49.563237,-70.042576,11.671184,275.212335,267.923668
0,2019-05-13,ADI,107.139999,107.680000,103.660004,96.640244,4900300.0,0.0,-0.249286,110.952114,99.932756,44.527712,-209.619942,31.042926,104.556204,101.852576
0,2019-05-13,ADP,158.410004,159.729996,157.000000,144.804428,1898900.0,0.0,0.054466,153.079861,144.267388,51.116722,-122.031909,12.592402,148.526603,144.703044


In [7]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 80, State Space: 801


In [8]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

In [9]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = None, risk_indicator_col= 'vix', **env_kwargs)

In [14]:
def env_creator_trading(env_config):
    # env_config is passed as {} and defaults are set here
    df = env_config.get('df', trade)  # changed from the training environment
    hmax = env_config.get('hmax', 100)
    initial_amount = env_config.get('initial_amount', 1000000)
    num_stock_shares = env_config.get('num_stock_shares', [0] * stock_dimension)
    buy_cost_pct = env_config.get('buy_cost_pct', buy_cost_list)
    sell_cost_pct = env_config.get('sell_cost_pct', sell_cost_list)
    state_space = env_config.get('state_space', 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension)
    stock_dim = env_config.get('stock_dim', stock_dimension)
    tech_indicator_list = env_config.get('tech_indicator_list', INDICATORS)
    action_space = env_config.get('action_space', stock_dimension)
    reward_scaling = env_config.get('reward_scaling', 1e-4)
    # specific for trading
    #turbulence_threshold = env_config.get('turbulence_threshold', 70)
    #risk_indicator_col = env_config.get('risk_indicator_col', 'vix')
    return EnvCompatibility(StockTradingEnv(
        df=df,
        hmax=hmax,
        initial_amount=initial_amount,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=buy_cost_pct,
        sell_cost_pct=sell_cost_pct,
        state_space=state_space,
        stock_dim=stock_dim,
        tech_indicator_list=tech_indicator_list,
        action_space=action_space,
        reward_scaling=reward_scaling,
        #turbulence_threshold=turbulence_threshold,
        #risk_indicator_col=risk_indicator_col
    ))

In [15]:
from ray.rllib.agents import ppo
#ray.shutdown()

In [16]:
config = ppo.PPOConfig()
config = config.training()
config = config.environment(env_config={'hmax':100, 'initial_amount':1000000})
config = config.framework(framework="torch")
# Rollouts
config = config.rollouts(num_rollout_workers=1,num_envs_per_worker=1,)
# Additional because we are evaluating now
config = config.exploration(explore=False)
config['seed'] = 0
config["model"]["fcnet_hiddens"] = [1024, 256, 128, 32]

In [17]:
# registering the environment to ray
register_env("finrl_trading", env_creator_trading)
#trainer = config.build(env="finrl")
# define trainer
#trainer = ppo.PPOTrainer(env='finrl_trading', config=config)

trainer = config.build(env="finrl_trading") 

(RolloutWorker pid=25687) 2023-04-02 07:37:23,991	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=25687) 2023-04-02 07:37:23,992	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
(RolloutWorker pid=25687) /home/ga_aiot/anaconda3/envs/finrl/lib/python3.8/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Casting input x to numpy array.
(RolloutWorker pid=25687)   logger.warn("Casting input x to numpy array.")
2023-04-02 07:37:24,920	INFO trainable.py:172 -- Trainable.setup took 10.532 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [19]:
# load saved agent
#cwd_checkpoint = f"results/checkpoints/ppo_{total_episodes+1}/checkpoint_0000{total_episodes+1}"
#print(f"Restoring from checkpoint {total_episodes+1}")
cwd_checkpoint = 'model/4fcnet/checkpoint_000400'
trainer.restore(cwd_checkpoint)

2023-04-02 07:38:16,203	INFO trainable.py:791 -- Restored on 172.28.238.49 from checkpoint: model/4fcnet/checkpoint_000400
2023-04-02 07:38:16,205	INFO trainable.py:800 -- Current state after restoring: {'_iteration': 400, '_timesteps_total': None, '_time_total': 24661.555459737778, '_episodes_total': 786}


In [20]:
import time 
def DRL_prediction(model, environment):
    start = time.time()
    """make a prediction"""
    state = environment.reset()
    for i in range(len(environment.df.index.unique())):
        action = model.compute_single_action(state)
        state, reward, done, _ = environment.step(action)
        if i == (len(environment.df.index.unique()) - 2):
            account_memory = environment.save_asset_memory()
            actions_memory = environment.save_action_memory()
        if done:
            break
    print("Test Finished!")
    print((time.time()-start),'s')
    return account_memory, actions_memory

In [ ]:
df_account_value.iloc[-1]

In [22]:
df_account_value, df_actions = DRL_prediction(
    model=trainer, 
    environment = e_trade_gym)
print(df_account_value)
df_actions.tail(15)


Test Finished!
12.164572477340698 s
           date   account_value
0    2019-05-13  1000000.000000
1    2019-05-14   789913.209833
2    2019-05-15   793368.915868
3    2019-05-16   801997.760331
4    2019-05-17   789936.895066
..          ...             ...
974  2023-03-24   890173.571299
975  2023-03-27   886046.208960
976  2023-03-28   877795.225988
977  2023-03-29   905659.602032
978  2023-03-30   919313.626297

[979 rows x 2 columns]


,AAL,AAPL,ADBE,ADI,ADP,ADSK,ALGN,AMAT,AMD,AMGN,...,TTWO,TXN,UAL,ULTA,VRSN,VRTX,WBA,WDC,WYNN,XEL
date,,,,,,,,,,,,,,,,,,,,,
2023-03-09,-38,0,0,0,0,0,0,-66,0,0,...,0,0,0,0,0,0,-68,0,-96,-1
2023-03-10,0,0,0,0,0,0,0,-27,0,-100,...,0,0,-68,0,0,0,100,0,0,0
2023-03-13,-60,0,0,0,0,0,0,-7,0,-28,...,0,0,0,-100,0,0,-32,0,0,0
2023-03-14,1,0,0,0,0,0,0,0,0,0,...,0,0,-29,0,2,0,4,0,0,0
2023-03-15,-1,0,0,0,0,0,0,0,0,-51,...,0,0,0,-4,0,0,0,0,0,0
2023-03-16,0,0,0,0,0,0,0,0,0,0,...,100,0,0,0,0,0,0,0,1,0
2023-03-17,0,0,0,0,0,0,0,0,0,-21,...,-9,0,0,0,0,0,0,0,0,0
2023-03-20,0,0,0,0,0,0,0,0,0,0,...,-91,0,0,0,-2,0,63,0,0,0
2023-03-21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,100,-100,0,-1,0


In [ ]:
#Total Trades
counts = (df_actions > 0).sum()
tdays = int(df_actions.shape[0])
print(f'Trades:{counts.sum()}\nTrade Days:{tdays}\nTrades Per Day:{counts.sum()/tdays:.2f}')

In [ ]:
df_actions.sum()

In [ ]:
import datetime
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./results/"+"perf_stats_all"+now+'.csv')

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_naxme = 'close')

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^NDX', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])